# Homework 10

In this homework, you will train a sentiment classifier on the [SST-2](https://huggingface.co/datasets/sst2) dataset using the pre-trained BERT model. For simplicity, I recommend using the [Hugging Face Transformers library](https://huggingface.co/docs/transformers/index). I've linked to corresponding tutorials below. You're welcome to use a different framework if you prefer.

# Problem 1

1. Fine-tune [DistilBERT](https://huggingface.co/distilbert-base-uncased) from scratch on SST-2 and evaluate the results. You can find a tutorial for loading BERT and fine-tuning [here](https://huggingface.co/docs/transformers/training). In that tutorial, you will need to change the dataset from `"yelp_review_full"` to `"sst2"` and the model from `"bert-base-uncased"` to `"distilbert-base-uncased"`. You'll also need to modify the code since SST-2 is a two-class classification dataset (unlike the Yelp Reviews dataset, which is a five-class classification dataset).

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install datasets

In [ ]:
!{sys.executable} -m pip install transformers

In [ ]:
!{sys.executable} -m pip install evaluate

**Load Dataset**

In [5]:
from datasets import load_dataset
dataset = load_dataset("sst2")

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset sst2 downloaded and prepared to /root/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
dataset["train"][100]

{'idx': 100, 'sentence': 'in memory ', 'label': 1}

**Tokenization**

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [8]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42)

**Train**

In [9]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

Training hyperparameters

In [10]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="test_trainer")

Evaluate

In [11]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

Trainer

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.370803,0.847477
2,No log,0.464819,0.840596
3,No log,0.563388,0.850917


TrainOutput(global_step=375, training_loss=0.26951810709635415, metrics={'train_runtime': 174.8675, 'train_samples_per_second': 17.156, 'train_steps_per_second': 2.144, 'total_flos': 397402195968000.0, 'train_loss': 0.26951810709635415, 'epoch': 3.0})

2. Choose a different pre-trained BERT-style model from the [Hugging Face Model Hub](https://huggingface.co/models) and fine-tune it. There are tons of options - part of the homework is navigating the hub to find different models! I recommend picking a model that is smaller than BERT-Base (as DistilBERT is) just to make things computationally cheaper. Is the final validation accuracy higher or lower with this other model?

In [16]:
import torch

del model
del trainer
torch.cuda.empty_cache()

In [17]:
dataset = load_dataset("sst2")
dataset["train"][100]

  0%|          | 0/3 [00:00<?, ?it/s]

{'idx': 100, 'sentence': 'in memory ', 'label': 1}

**Tokenization**

In [18]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [19]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42)

**Train**

In [20]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [21]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="test_trainer")

Evaluate

In [22]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [23]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

Trainer

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.614589,0.711009
2,No log,0.330100,0.883028
3,No log,0.424832,0.880734


TrainOutput(global_step=375, training_loss=0.5162443440755208, metrics={'train_runtime': 340.9953, 'train_samples_per_second': 8.798, 'train_steps_per_second': 1.1, 'total_flos': 789333166080000.0, 'train_loss': 0.5162443440755208, 'epoch': 3.0})

# Problem 2

Instead of fine-tuning the full model on a target dataset, it's also possible to use the output representations from a BERT-style model as input to a linear classifier and *only* train the classifier (leaving the rest of the pre-trained parameters fixed). You can do this easily using the [`sentence-transformers`](https://www.sbert.net/) library. Using `sentence-tranformers` gives you back a fixed-length representation of a given text sequence. To achieve this, you need to 
1. Pick a pre-trained sentence Transformer.
2. Load the SST-2 dataset and feed the text from each example into the model.
3. Train a linear classifier on the representations.
4. Evaluate performance on the validation set.

For the second step, you can learn more about how to use Hugging Face datasets [here](https://huggingface.co/docs/datasets/index). For the third and fourth step, you can do this directly in PyTorch, or you can just collect the learned representations and use them as feature vectors to train a linear classifier in any other library (e.g. [scikit-learn](https://scikit-learn.org/stable/modules/linear_model.html).

After you complete the above steps, report whether the accuracy on the validation set is higher or lower using a fixed sentence Transformer.

In [ ]:
!pip install sentence-transformers

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
from sklearn.metrics import accuracy_score

In [28]:
del model
torch.cuda.empty_cache()

**Pre-trained sentence Transformer**

In [29]:
# Load the pre-trained sentence Transformer
model = 'sentence-transformers/paraphrase-distilroberta-base-v2'
sentence_transformer = SentenceTransformer(model)

**Load the data**

In [30]:
# Load the SST-2 dataset
dataset = load_dataset('sst2')
train_data = dataset['train']
validation_data = dataset['validation']

  0%|          | 0/3 [00:00<?, ?it/s]

**Encode the text**

In [31]:
# Encode the text from each example in the train and validation sets
train_embeddings = sentence_transformer.encode(train_data['sentence'], convert_to_tensor=True)
validation_embeddings = sentence_transformer.encode(validation_data['sentence'], convert_to_tensor=True)

train_labels = torch.tensor(train_data['label'])
validation_labels = torch.tensor(validation_data['label'])

# Move embeddings and labels to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_embeddings, train_labels = train_embeddings.to(device), train_labels.to(device)
validation_embeddings, validation_labels = validation_embeddings.to(device), validation_labels.to(device)

**Train and Evaluate**

In [32]:
# Create a simple logistic regression model using PyTorch
input_size = train_embeddings.size(1)
num_classes = 2
logistic_regression = nn.Linear(input_size, num_classes).to(device)

In [33]:
# Set the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(logistic_regression.parameters(), lr=0.01)

# Train the logistic regression model and print accuracy for each epoch
num_epochs = 50
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = logistic_regression(train_embeddings)
    loss = criterion(outputs, train_labels)
    loss.backward()
    optimizer.step()

    # Compute training accuracy
    with torch.no_grad():
        _, train_predicted = torch.max(outputs, 1)
        train_accuracy = accuracy_score(train_labels.cpu(), train_predicted.cpu())

    # Compute validation accuracy
    with torch.no_grad():
        validation_outputs = logistic_regression(validation_embeddings)
        _, validation_predicted = torch.max(validation_outputs, 1)
        validation_accuracy = accuracy_score(validation_labels.cpu(), validation_predicted.cpu())

    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Train Accuracy: {train_accuracy:.4f}, '
          f'Validation Accuracy: {validation_accuracy:.4f}')

Epoch [1/50], Train Accuracy: 0.4480, Validation Accuracy: 0.5275
Epoch [2/50], Train Accuracy: 0.4951, Validation Accuracy: 0.5550
Epoch [3/50], Train Accuracy: 0.5451, Validation Accuracy: 0.5883
Epoch [4/50], Train Accuracy: 0.5878, Validation Accuracy: 0.6112
Epoch [5/50], Train Accuracy: 0.6217, Validation Accuracy: 0.6296
Epoch [6/50], Train Accuracy: 0.6502, Validation Accuracy: 0.6468
Epoch [7/50], Train Accuracy: 0.6732, Validation Accuracy: 0.6651
Epoch [8/50], Train Accuracy: 0.6938, Validation Accuracy: 0.6778
Epoch [9/50], Train Accuracy: 0.7109, Validation Accuracy: 0.6938
Epoch [10/50], Train Accuracy: 0.7258, Validation Accuracy: 0.7053
Epoch [11/50], Train Accuracy: 0.7382, Validation Accuracy: 0.7122
Epoch [12/50], Train Accuracy: 0.7501, Validation Accuracy: 0.7225
Epoch [13/50], Train Accuracy: 0.7602, Validation Accuracy: 0.7339
Epoch [14/50], Train Accuracy: 0.7686, Validation Accuracy: 0.7420
Epoch [15/50], Train Accuracy: 0.7765, Validation Accuracy: 0.7500
Epoc

As we can see, the accuracy on the validation set is lower when using a fixed sentence Transformer compared to fine-tuning the entire model. During the fine-tuning process, all parameters are updated, allowing the model to become more specialized for the target task. Consequently, fine-tuning typically leads to better performance. However, it's worth noting that fine-tuning requires more computing resources than using a fixed sentence Transformer method.